# 01 - Enumerations

#### Motivation

How do we deal with collections of related constants? One way would be:

In [3]:
RED = 1
GREEN = 2
BLUE = 3

COLOURS = (RED, GREEN, BLUE)

RED in COLOURS

True

But this causes the following issues:

In [4]:
print(1 in COLOURS)
print(RED < GREEN)

True
True


Even if we were to assign the symbols to meaningful strings e.g. `RED = 'red'`, we can run into issues and accidents such as having non-unique values by doing `GREEN = 'red'` for example. We could also do `RED * 2`...

What do we really want? 

An **immutable** collection of related **constant** members:
- having unique names e.g. RED
- having an associated **constant** value e.g. 1
- having unique associated values
- not allowing operations e.g. `RED * 2`
- lookup member by name
- lookup member by value


**Aliases**

Sometimes we want multiple symbols to refer to the same thing:
```python
POLY_4 = 4
RECTANCLE = 4
SQUARE = 4
RHOMBUS = 4
```
Here, the last three are aliases for `POLY_4`. With reverse lookups, we want to return the original, not the alias, so `4 -> POLY_4`.

#### Lecture

Here's the basic setup:

In [5]:
from enum import Enum

class Colour(Enum):
    RED = 1
    GREEN = 2
    BLUE = 3

Here's some terminology:

- `Colour` is called an **enumeration** (enum for short)
- `Colour.RED` is called an **enumeration member** (and their values are just called values)
- The `type` of a member is the **enumeration** that it belongs to. This is odd; see below (and metaprogramming sections)

In [6]:
print(type(Colour.RED))

<enum 'Colour'>


In [7]:
class Colour2:
    RED = 1

type(Colour2.RED)

int

Here's some basics:

In [8]:
print(f"{isinstance(Colour.RED, Colour) = }")
print(f"{str(Colour.RED) = }")
print(f"{repr(Colour.RED) = }")
print(f"{Colour.RED.name = }")
print(f"{Colour.RED.value = }")


isinstance(Colour.RED, Colour) = True
str(Colour.RED) = 'Colour.RED'
repr(Colour.RED) = '<Colour.RED: 1>'
Colour.RED.name = 'RED'
Colour.RED.value = 1


**Equality and Membership**

Equality is done using identity:`is` (but `==` works too, but shouldn't be used as it can be overridden)

In [9]:
Colour.GREEN in Colour

True

But note that members and their associated values are **not equal**!

In [10]:
Colour.RED == 1

False

**Hashability**

Enumeration members are always **hashable**. This is so that they can be used as:
- keys in dictionaries
- elements of a set

For example:

In [11]:
pixel_colour = {
    Colour.RED: 100,
    Colour.GREEN: 25,
    Colour.BLUE: 255
}

**Lookup**

There are two ways we can retrieve a member. Either through its value directly (e.g. 1 for `Colour.RED`) or by the string of the member name (e.g. `'RED'`).

To do this, know that enumerations are **callables**.

Searching by value:

In [12]:
Colour(1)

<Colour.RED: 1>

Searching by member (note square brackets):

In [13]:
Colour['RED']

<Colour.RED: 1>

This works because `__getitem__` has been implemented.

We can use dot notation or `getattr` to search by member name too, if we want to avoid try-excepts when getting an enumeration member:

In [27]:
print(getattr(Colour, "TURQUOISE", None))

None


**Enumerating Members**

Enumerations are **iterables**, so we can `list(Colour)` with **definition order** preserved:

In [14]:
class Colour(Enum):
    GREEN = 2
    RED = 1
    BLUE = 3

list(Colour)

[<Colour.GREEN: 2>, <Colour.RED: 1>, <Colour.BLUE: 3>]

We also have the `__members__` property which returns a dict where the keys are the names as strings, and the values are the values.

In [15]:
Colour.__members__

mappingproxy({'GREEN': <Colour.GREEN: 2>,
              'RED': <Colour.RED: 1>,
              'BLUE': <Colour.BLUE: 3>})

**Constant Members**

Once an enumeration has been declared:
- member list is immutable (cannot add or remove members)
- member values are immutable
- cannot be subclassed, **unless it contains no members**. You'll see why creating empty enumerations can be useful.

In [22]:
Colour(1)

<Colour.RED: 1>

# 02 - Aliases

While members must be unique, the `Enum` class *does* let us create multiple members with the same value. The difference is, only the first one will be the *true* member and the rest will be **aliases**. 

This is allowed:

In [28]:
class Colour(Enum):
    red = 1
    crimson = 1
    carmine = 1
    blue = 2
    aquamarine = 2

list(Colour)

[<Colour.red: 1>, <Colour.blue: 2>]

The aliased members simply point to the original:

In [30]:
Colour.crimson

<Colour.red: 1>

The same is true for reverse lookups:

In [32]:
Colour(1)

<Colour.red: 1>

**Ensuring unique values**

Sometimes we may want a guarantee of no aliases. The easiest way to do this is with the `@enum.unique` decorator:

In [31]:
import enum

@enum.unique
class Colour(Enum):
    red = 1
    crimson = 1
    carmine = 1
    blue = 2
    aquamarine = 2

ValueError: duplicate values found in <enum 'Colour'>: crimson -> red, carmine -> red, aquamarine -> blue

#### Example

There are times when the ability to define these aliases can be useful. Let's say you have to deal with statuses that are returned as strings from different systems.

These systems may not always define exactly the same strings to mean the same thing (maybe they were developed independently). In a case like this, being able to create aliases could be useful to bring uniformity to our own code.

Let's say we have the following status definitions as described by different systems:

```
Us        System 1               System 2
-------------------------------------------
ready     ready                  ready
running   busy                   processing
ok        finished_no_error      ran_ok
errors    finished_with_errors   errored
```

We can the easily achieve this using this class with aliases:

In [34]:
class Status(enum.Enum):
    ready = 'ready'
    
    running = 'running'
    busy = 'running'
    processing = 'running'
    
    ok = 'ok'
    finished_no_error = 'ok'
    ran_ok = 'ok'
    
    errors = 'errors'
    finished_with_errors = 'errors'
    errored = 'errors'

In [36]:
list(Status)

[<Status.ready: 'ready'>,
 <Status.running: 'running'>,
 <Status.ok: 'ok'>,
 <Status.errors: 'errors'>]

We can now programmatically evaluate any status and have it return our definition:

In [37]:
Status['busy']

<Status.running: 'running'>

# 03 - Customizing and Extending Enumerations

#### Customising

Remember that enumerations are **classes** while enumeration members are class attributes that *become* **instances** of that class via metaprogramming.

This means that if we define functions in the enumeration class, these will be **bound methods** to the members. We can also implement dunder methods such as `__bool__`.

Consider the following example:

In [52]:
class State(Enum):
    READY = 1
    BUSY = 0

if State.READY:
    print('READY! Doing something...')

if State.BUSY:
    print('BUSY! Dont do something...')

READY! Doing something...
BUSY! Dont do something...


Out the box, **members are always truthy**, which is why `State.BUSY` returned `True`. We can override this with `__bool__`.

In [53]:
class State(Enum):
    READY = 1
    BUSY = 0

    def __bool__(self):
        return bool(self.value)

if State.READY:
    print('READY! Doing something...')

if State.BUSY:
    print('BUSY! Dont do something...')

READY! Doing something...


#### Extending/Subclassing

As mentioned before, we cannot extend an enum that contains members because the subclassed enum will be mutating the original which violates the innate immutability of enumerations.

But subclass an empty enumeration can be useful. Here we create a base enumeration class containing all the methods and none of the members.

Then we subclass it and create our members to have all the methods and all the members.

##### Simple Example

Here's an example of where this might be useful:

In [57]:
from functools import total_ordering

@total_ordering
class OrderedEnum(Enum):
    """Creates an ordering based on the member values. 
    So member values have to support rich comparisons.
    """
    
    def __lt__(self, other):
        if isinstance(other, OrderedEnum):
            return self.value < other.value
        return NotImplemented

And now we can create other enumerations that will support ordering without having to retype the `__lt__` implementation, or even the decorator:

In [58]:
class Number(OrderedEnum):
    ONE = 1
    TWO = 2
    THREE = 3
    
class Dimension(OrderedEnum):
    D1 = 1,
    D2 = 1, 1
    D3 = 1, 1, 1

In [59]:
Number.ONE < Number.THREE

True

In [60]:
Dimension.D1 < Dimension.D3

True

##### HTTP Example

The HTTP status codes are a type of enum (called `EnumMeta` but don't worry about that): 

In [62]:
from http import HTTPStatus

list(HTTPStatus)[0:5]

[<HTTPStatus.CONTINUE: 100>,
 <HTTPStatus.SWITCHING_PROTOCOLS: 101>,
 <HTTPStatus.PROCESSING: 102>,
 <HTTPStatus.EARLY_HINTS: 103>,
 <HTTPStatus.OK: 200>]

In [63]:
HTTPStatus(404)

<HTTPStatus.NOT_FOUND: 404>

This has other properties besides `<member>.name` and `<member>.value` such as `<member>.phrase` which is more readable:

In [67]:
HTTPStatus.NOT_FOUND.name, HTTPStatus.NOT_FOUND.value, HTTPStatus.NOT_FOUND.phrase, 

('NOT_FOUND', 404, 'Not Found')

#### Replicating HTTP Example

Let's make a simpler version of `HTTPStatus` with only two statuses.

To do this, there are a few things we need to remember:
- members **need** to be **instances** of our class.
- `__new__` is responsible for creating instances and, **for Enums**, it runs when compiling the class. Its first argument is always `cls`.
- The remaining arguments of `__new__` are our members' declared values when unpacked. If we have a single value, we have a single additional arg. If we have a tuple of three values, we have three additional args.
- enumeration members have an special `._value_` attribute inherited from `Enum` which is used to map our declared value to `<member>.value`.

With all that being said, we can create our replicate:

In [91]:
class AppStatus(Enum):
    OK = (0, 'No Problem!')
    FAILED = (1, 'Crap!')
    
    def __new__(cls, member_value, member_phrase):
        # create a new instance of cls
        print(cls, member_value, member_phrase)
        member = object.__new__(cls)
        
        # set up instance attributes
        member._value_ = member_value
        member.phrase = member_phrase
        return member

iran
new ran
<enum 'AppStatus'> 0 No Problem!
new ran
<enum 'AppStatus'> 1 Crap!


We needed to delegate to `object` for instantiating (not initialising) our instance of `AppStatus` as opposed to instantiating via `member = AppStatus()` as that would implictly call `__new__` and cause infinite recursion.

Thereafter, we setup our own attributes on our member.

This gives us the replicated functionality of `HTTPStatus`:

In [78]:
AppStatus.OK.name, AppStatus.OK.value, AppStatus.OK.phrase

('OK', 0, 'No Problem!')

In [79]:
AppStatus.FAILED.name, AppStatus.FAILED.value, AppStatus.FAILED.phrase

('FAILED', 1, 'Crap!')

This is a strong case for subclassing:

In [93]:
class TwoValueEnum(Enum):
    def __new__(cls, member_value, member_phrase):
        member = object.__new__(cls)
        member._value_ = member_value
        member.phrase = member_phrase
        return member

And then inherit this for any enumeration where we want to support a value as a `(code, phrase)` tuple:

In [94]:
class AppStatus(TwoValueEnum):
    OK = (0, 'No Problem!')
    FAILED = (1, 'Crap!')

In [95]:
AppStatus.OK.name, AppStatus.OK.value, AppStatus.OK.phrase

('OK', 0, 'No Problem!')

In [97]:
AppStatus(0)

<AppStatus.OK: 0>

# 04 - Automatic Values

We can use `enum.auto()` to assign values to members automatically. 

Behind the scenes, this uses a static method in the `Enum` class called `_generate_next_value(name, start, count, last_values)`.
- `name`: the member's name
- `start`: we won't cover this; it's essentially used for creating enumerations functionally as opposed to declaratively
- `count`: the number of members that have **already** been created (including aliases!)
- `last_values`: a list containing all the previous values that have been assigned to members.

returns: value to be assigned to that particular member.

By default, its implementation results in sequential integer numbers from 1.

Here's a basic example:

In [103]:
class Number(Enum):
    ONE = enum.auto()
    TWO = enum.auto()
    THREE = enum.auto()

list(Number)

[<Number.ONE: 1>, <Number.TWO: 2>, <Number.THREE: 3>]